In [1]:
import json
import os

import devlib
from wlgen import RTA

import trappy

In [2]:
# Configure logging for this session
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    level=logging.DEBUG,
    datefmt='%I:%M:%S')

# Global configuration

In [3]:
# Host side results folder
RESULTS_DIR = '/tmp/schedtest'

# Taerget side temporary folder
TARGET_DIR = '/root/schedtest'

# List of tools to install on the target system
TOOLS = ["rt-app", "trace-cmd", "taskset"]

# FTrace Configuration
FTRACE_EVENTS = ['sched_switch', 'cpu_frequency']
FTRACE_BUFFZISE = 10240

# HWMon Configuration
HWMON_CONF = {
    'sites' : [ 'a53', 'a57' ],
    'kinds' : [ 'energy' ]
}

# List of modules to enable
MODULES = ['bl', 'cpufreq', 'hwmon']

# Target configuration

In [4]:
target = devlib.LinuxTarget(
    connection_settings={
        'host'     : '192.168.0.10',
        'username' : 'root',
        'password' : ''
    },
    load_default_modules=False,
    modules=MODULES,
    working_directory=TARGET_DIR
)

01:44:18  DEBUG   : Installing module bl
01:44:19  DEBUG   : Installing module cpufreq
01:44:19  DEBUG   : Installing module hwmon


In [5]:
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

01:44:47  INFO    : Target ABI: arm64, CPus: ['A53', 'A57', 'A57', 'A53', 'A53', 'A53']


In [6]:
logging.info('Copy tools required on target')
tools_to_install = []
for tool in TOOLS:
    # deploy scripts or...
    binary = '../tools/scripts/{}'.format(tool)
    if not os.path.isfile(binary):
        # ... binary tool matching the target ABI
        binary = '../tools/{}/{}'.format(target.abi, tool)
    tools_to_install.append(binary)
target.setup(tools_to_install)

01:44:47  INFO    : Copy tools required on target


# FTrace configuration

In [7]:
ftrace = None
if len(FTRACE_EVENTS):
    ftrace = devlib.FtraceCollector(
        target,
        events = FTRACE_EVENTS,
        buffer_size = FTRACE_BUFFZISE,
        autoreport = False,
        autoview = False
    )
    logging.info('FTrace configured to collect events:')
    logging.info('  %s', FTRACE_EVENTS)
else:
    logging.info('FTrace collection disabled by configuration')
    
def ftrace_start():
    if ftrace is None:
        return
    ftrace.start()
    logging.info('FTrace STARTED')
    
def ftrace_stop():
    if ftrace is None:
        return
    ftrace.stop()
    logging.info('FTrace STOPPED')
    ftrace.get_trace(RESULTS_DIR + '/trace.dat')

01:44:50  INFO    : FTrace configured to collect events:
01:44:50  INFO    :   ['sched_switch', 'cpu_frequency']


# Energy Meter configuration

In [8]:
hwmon = None
if 'hwmon' in MODULES:
    hwmon = devlib.HwmonInstrument(target)
    hwmon.reset(**HWMON_CONF)
    logging.info('Channels selected for energy sampling:')
    logging.info('  %s', str(hwmon.active_channels))
else:
    logging.info('Energy sampling disabled by configuration')
    
# The contained for sampled energy values
energy_reading = {}

def hwmon_reset():
    energy_reading = {}

# Simple function to compute energy-deltas among consecutive readings
def hwmon_sample():
    if hwmon is None:
        return
    samples = hwmon.take_measurement()
    logging.debug('Measure: %s', samples)
    for s in samples:
        label = s.channel.label\
                    .replace('_energy', '')\
                    .replace(" ", "_")
        value = s.value
        logging.debug('Update %s: %s',
                      label, value)

        if label not in energy_reading:
            energy_reading[label] = {
                 'last'  : value,
                 'delta' : 0,
                 'total' : 0
                }
            logging.debug('Initialize %s: %s',
                          label, energy_reading[label])
            continue

        last  = energy_reading[label]['last']
        delta = value - last
        total = energy_reading[label]['total']

        energy_reading[label]['last']  = value
        energy_reading[label]['delta'] = delta
        energy_reading[label]['total'] = total + delta
        
        logging.debug('Update %s: %s',
                          label, energy_reading[label])

    return energy_reading


01:44:51  DEBUG   : Discovering available HWMON sensors...
01:44:51  DEBUG   : 	Adding sensor v2m_juno_amp/curr1
01:44:51  DEBUG   : 	Adding sensor v2m_juno_amp/curr1
01:44:51  DEBUG   : 	Adding sensor v2m_juno_power/power1
01:44:51  DEBUG   : 	Adding sensor v2m_juno_power/power1
01:44:51  DEBUG   : 	Adding sensor v2m_juno_energy/energy1
01:44:51  DEBUG   : 	Adding sensor v2m_juno_energy/energy1
01:44:51  DEBUG   : 	Adding sensor v2m_juno_energy/energy1
01:44:51  DEBUG   : 	Adding sensor v2m_juno_energy/energy1
01:44:51  DEBUG   : 	Adding sensor v2m_juno_amp/curr1
01:44:51  DEBUG   : 	Adding sensor v2m_juno_amp/curr1
01:44:51  DEBUG   : 	Adding sensor v2m_juno_volt/in1
01:44:51  DEBUG   : 	Adding sensor v2m_juno_volt/in1
01:44:51  DEBUG   : 	Adding sensor v2m_juno_volt/in1
01:44:51  DEBUG   : 	Adding sensor v2m_juno_volt/in1
01:44:51  DEBUG   : 	Adding sensor v2m_juno_power/power1
01:44:51  DEBUG   : 	Adding sensor v2m_juno_power/power1
01:44:51  INFO    : Channels selected for energy 

# Workload configuration

In [9]:
# RTApp calibration

# Uncomment the following line to calibrate RTApp the first time
# target_calibration = RTA.calibrate(target)

# Provide a pre-defined calibration
target_calibration = {0: 353, 1: 138, 2: 138, 3: 353, 4: 354, 5: 360}

In [10]:
rtapp = RTA(target, 'simple', calibration=target_calibration)
rtapp.conf(
    kind='profile',
    params={
        'task1': RTA.periodic(
            period_ms=100,
            duty_cycle_pct=20,
            duration_s=5)
    },
    run_dir=TARGET_DIR
);

01:44:51  ERROR   : Assuming taskset is preinstalled
01:44:51  INFO    : Setup new workload simple
01:44:51  DEBUG   : Setup step [postrun] callback to [__postrun] function
01:44:51  DEBUG   : Configuring a profile-based workload...
01:44:51  DEBUG   : ref on big cpu: 1
01:44:51  INFO    : Workload duration defined by longest task
01:44:51  INFO    : ------------------------
01:44:51  INFO    : task [task1], SCHED_OTHER:
01:44:51  INFO    :  | loops count: 1
01:44:51  INFO    :  + phase_000001: duration 5.000000 [s] (50 loops)
01:44:51  INFO    :  |  period   100000 [us], duty_cycle  20 %
01:44:51  INFO    :  |  run_time  20000 [us], sleep_time  80000 [us]


# Workload execution

In [11]:
logging.info('Create output folder')
os.system('mkdir {}'.format(RESULTS_DIR));

01:44:51  INFO    : Create output folder


In [12]:
hwmon_reset()

ftrace_start()
hwmon_sample()

logging.info('RTApp STARTING...')
rtapp.run(out_dir=RESULTS_DIR)

nrg = hwmon_sample();
ftrace_stop()

01:44:58  INFO    : FTrace STARTED
01:44:59  DEBUG   : Measure: [a53_energy: 11588.717062 joules, a57_energy: 16230.437033 joules]
01:44:59  DEBUG   : Update a53: 11588.717062
01:44:59  DEBUG   : Initialize a53: {'total': 0, 'last': 11588.717062, 'delta': 0}
01:44:59  DEBUG   : Update a57: 16230.437033
01:44:59  DEBUG   : Initialize a57: {'total': 0, 'last': 16230.437033, 'delta': 0}
01:44:59  INFO    : RTApp STARTING...
01:44:59  INFO    : Executor [start]: /root/schedtest/bin/rt-app /root/schedtest/simple_00.json
01:45:04  DEBUG   : Callback [postrun]...
01:45:04  DEBUG   : Pulling logfiles to [/tmp/schedtest]...
01:45:04  DEBUG   : Pulling JSON to [/tmp/schedtest]...
01:45:04  DEBUG   : Saving output on [/tmp/schedtest/output.log]...
01:45:04  INFO    : Executor [end]: /root/schedtest/bin/rt-app /root/schedtest/simple_00.json
01:45:05  DEBUG   : Measure: [a53_energy: 11589.953522 joules, a57_energy: 16240.57781 joules]
01:45:05  DEBUG   : Update a53: 11589.953522
01:45:05  DEBUG   :

In [13]:
logging.info('Energy: %s',
    json.dumps(nrg,
                indent=4,
                separators=(',', ': ')))

01:45:09  INFO    : Energy: {
    "a53": {
        "total": 1.2364600000000792,
        "last": 11589.953522,
        "delta": 1.2364600000000792
    },
    "a57": {
        "total": 10.140777000000526,
        "last": 16240.57781,
        "delta": 10.140777000000526
    }
}


# Trace inspection

In [14]:
trappy.plotter.plot_trace(RESULTS_DIR)